In [1]:
import numpy as np
import plotly.graph_objects as go 
import pyrender 
import open3d as o3d
import trimesh

## Superquadrics

A thorough overview of superquadrics: https://cse.buffalo.edu/~jryde/cse673/files/superquadrics.pdf

### Implicit Equation
The surface of a basic superquadric is given by: 
$$ \vert x \vert^r + \vert y \vert^s + \vert z \vert^t = 1 ,$$

where $r, s$ and $t$ are positive real numbers that determine the main features of the superquadric:
 - $< 1$ : pointy octahedron
 - $= 1$: regular octahedron
 - $range(1, 2)$: blunt octahedron
 - $=2$ : sphere
 - $>2$ : cube modified to have rounded edges and corners
 
 We can scale these basic shapes as follows: 
 
 $$ \vert \frac{x}{A} \vert^r + \vert \frac{y}{B} \vert^s + \vert \frac{z}{C} \vert^t = 1 $$


### Parametric description

We can describe a superquadric with a parametric equation in terms of surface parameters u (longitude) and v (latitude) by:

\begin{align}
& x(u,v) = Ag\left(v, \frac{2}{r}\right)g\left(u, \frac{2}{r}\right) \\
& y(u,v) = Bg\left(v, \frac{2}{s}\right)f\left(u, \frac{2}{s}\right) \\
& z(u,v) = Cf\left(v, \frac{2}{t}\right) \\
& -\frac{\pi}{2} \leq v \leq \frac{\pi}{2}, \quad -\pi \leq u \leq \pi
\end{align}

The auxilary functions $f$ and $g$ are:

\begin{align}
& f(\omega, m) = sgn(\sin \omega) \vert \sin \omega \vert^m \\
& f(\omega, m) = sgn(\cos \omega) \vert \cos \omega \vert^m \\
\end{align}

where $sgn(x)$ is the sign function (-1 if x < 0, 0 if x == 0, 1 if x > 0) 

In [107]:
def fexp(omega, m):
    return np.sign(omega) * np.power(np.abs(omega), m)


def get_eta_and_w(n, eta_max=np.pi/2, w_max=np.pi):
    '''
    Taken from: https://github.com/HawaiiZeng/Superquadrics/blob/master/superquadrics.py
    '''
    eta_min = -eta_max
    w_min = -w_max
    d_w = (w_max - w_min) / n
    
    # When the range = 2PI, one point will be overlapped
    x = np.linspace(0, n, n + 1)
    if eta_max == np.pi / 2:
        n = n - 1
    y = np.linspace(0, n, n + 1)
    
    d_eta = (eta_max - eta_min) / n

    yv, xv = np.meshgrid(y, x)
    eta = eta_min + yv * d_eta
    w = w_min + xv * d_w
    return eta, w


def supertoroid(epsilon, A, n=50):
    '''
        ref: https://cse.buffalo.edu/~jryde/cse673/files/superquadrics.pdf EQ 2.22
        A is now a list of 4 scaling factors
    '''
    eta, w = get_eta_and_w(n, eta_max=np.pi, w_max=np.pi)
    
    x = A[0] * (A[3] + fexp(np.cos(eta), epsilon[0])) * fexp(np.cos(w), epsilon[0])
    y = A[1] * (A[3] + fexp(np.cos(eta), epsilon[1])) * fexp(np.sin(w), epsilon[1])
    z = A[3] * fexp(np.sin(eta), epsilon[2])
    
    return x,y,z
    
def superellipsoid(epsilon, A, n=50):
    '''
    This follows the equation from: https://en.wikipedia.org/wiki/Superquadrics
    epsilon is a 3-element vector: 2/r, 2/s, 2/t
    A is a 3-element vector: A, B, C
    '''
    eta, w = get_eta_and_w(n)
    
    x = A[0] * fexp(np.sin(eta), epsilon[0]) * fexp(np.sin(w), epsilon[0])
    y = A[1] * fexp(np.sin(eta), epsilon[1]) * fexp(np.cos(w), epsilon[1])
    z = A[2] * fexp(np.cos(eta), epsilon[2])
    
    return x,y,z

In [108]:
def save_to_obj(save_path, x, y, z):
    """
    This function saves super-ellpsoid w/o overlap: the rightmost vertices are not coincide with the leftmost points,
    and only one vertex at the top and bottom
    """
    x = np.transpose(x, (1, 0))
    y = np.transpose(y, (1, 0))
    z = np.transpose(z, (1, 0))
    print(x.shape, y.shape, z.shape)
    hei, wid = x.shape[0], x.shape[1]
    count = 0
    with open(save_path, "w+") as fout:
        # vertices of body
        for i in range(1, hei - 1):
            for j in range(0, wid - 1):
                fout.write("v %.3f %.3f %.3f\n" % (x[i, j], y[i, j], z[i, j]))
                count += 1
        # faces
        for i in range(0, hei - 3):
            for j in range(0, wid - 2):
                fout.write("f %d %d %d %d\n" % (i * (wid - 1) + j + 1, i * (wid - 1) + j + 2, (i + 1) * (wid - 1) + j + 2, (i + 1) * (wid - 1) + j + 1))
        for i in range(0, hei - 3):
            fout.write("f %d %d %d %d\n" % (i * (wid - 1) + wid - 2 + 1, i * (wid - 1) + 0 + 1, (i + 1) * (wid - 1) + 0 + 1, (i + 1) * (wid - 1) + wid - 2 + 1))
        fout.write("v %.3f %.3f %.3f\n" % (x[0, 0], y[0, 0], z[0, 0]))
        fout.write("v %.3f %.3f %.3f\n" % (x[-1, -1], y[-1, -1], z[-1, -1]))
        for j in range(0, wid - 2):
            fout.write("f %d %d %d\n" % (count+1, j + 2, j + 1))
        fout.write("f %d %d %d\n" % (count+1, 1, wid - 2 + 1))
        for j in range(0, wid - 2):
            fout.write("f %d %d %d\n" % (count+2, (hei - 3) * (wid - 1) + j + 1, (hei - 3) * (wid - 1) + j + 2))
        fout.write("f %d %d %d\n" % (count+2, (hei - 3) * (wid - 1) + wid - 3 + 2, (hei - 3) * (wid - 1) + 1))


In [109]:
epsilon = [2, 1, 1]
scaling = [1,1,1]

x,y,z = superellipsoid(epsilon, scaling)
save_to_obj('objects/superellipsoid.obj', x, y, z)

x,y,z = supertoroid(epsilon, [1,1,1,1], n=200)
save_to_obj('objects/supertoroid.obj', x, y, z)


(50, 51) (50, 51) (50, 51)
(201, 201) (201, 201) (201, 201)


In [106]:
s_toroid = trimesh.load('objects/supertoroid.obj')
s_ellipsoid = trimesh.load('objects/superellipsoid.obj')

toroid_mesh = pyrender.Mesh.from_trimesh(s_toroid)
ellipsoid_mesh = pyrender.Mesh.from_trimesh(s_ellipsoid)

scene = pyrender.Scene()
#scene.add(toroid_mesh)
scene.add(ellipsoid_mesh)
pyrender.Viewer(scene, use_raymond_lighting=True)

WARNING - 2020-11-03 16:25:52,840 - obj - faces have mixed data, using slow fallback!
WARNING - 2020-11-03 16:25:53,175 - obj - faces have mixed data, using slow fallback!


Viewer(width=640, height=480)